<a href="https://colab.research.google.com/github/aashitadutta/ADM_Project2/blob/master/ADM_Descritization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import statments

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Reading the data and names file

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/ADM Project/communities.data", header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127
0,8,?,?,Lakewoodcity,1,0.19,0.33,0.02,0.90,0.12,0.17,0.34,0.47,0.29,0.32,0.20,1.0,0.37,0.72,0.34,0.60,0.29,0.15,0.43,0.39,0.40,0.39,0.32,0.27,0.27,0.36,0.41,0.08,0.19,0.10,0.18,0.48,0.27,0.68,0.23,...,0.35,0.38,0.34,0.38,0.46,0.25,0.04,0.0,0.12,0.42,0.50,0.51,0.64,0.03,0.13,0.96,0.17,0.06,0.18,0.44,0.13,0.94,0.93,0.03,0.07,0.1,0.07,0.02,0.57,0.29,0.12,0.26,0.20,0.06,0.04,0.9,0.5,0.32,0.14,0.20
1,53,?,?,Tukwilacity,1,0.00,0.16,0.12,0.74,0.45,0.07,0.26,0.59,0.35,0.27,0.02,1.0,0.31,0.72,0.11,0.45,0.25,0.29,0.39,0.29,0.37,0.38,0.33,0.16,0.30,0.22,0.35,0.01,0.24,0.14,0.24,0.30,0.27,0.73,0.57,...,0.38,0.40,0.37,0.29,0.32,0.18,0.00,0.0,0.21,0.50,0.34,0.60,0.52,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,0.02,0.12,0.45,?,?,?,?,0.00,?,0.67
2,24,?,?,Aberdeentown,1,0.00,0.42,0.49,0.56,0.17,0.04,0.39,0.47,0.28,0.32,0.00,0.0,0.30,0.58,0.19,0.39,0.38,0.40,0.84,0.28,0.27,0.29,0.27,0.07,0.29,0.28,0.39,0.01,0.27,0.27,0.43,0.19,0.36,0.58,0.32,...,0.29,0.27,0.31,0.48,0.39,0.28,0.00,0.0,0.14,0.49,0.54,0.67,0.56,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,0.01,0.21,0.02,?,?,?,?,0.00,?,0.43
3,34,5,81440,Willingborotownship,1,0.04,0.77,1.00,0.08,0.12,0.10,0.51,0.50,0.34,0.21,0.06,1.0,0.58,0.89,0.21,0.43,0.36,0.20,0.82,0.51,0.36,0.40,0.39,0.16,0.25,0.36,0.44,0.01,0.10,0.09,0.25,0.31,0.33,0.71,0.36,...,0.70,0.77,0.89,0.63,0.51,0.47,0.00,0.0,0.19,0.30,0.73,0.64,0.65,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,0.02,0.39,0.28,?,?,?,?,0.00,?,0.12
4,42,95,6096,Bethlehemtownship,1,0.01,0.55,0.02,0.95,0.09,0.05,0.38,0.38,0.23,0.36,0.02,0.9,0.50,0.72,0.16,0.68,0.44,0.11,0.71,0.46,0.43,0.41,0.28,0.00,0.74,0.51,0.48,0.00,0.06,0.25,0.30,0.33,0.12,0.65,0.67,...,0.36,0.38,0.38,0.22,0.51,0.21,0.00,0.0,0.11,0.72,0.64,0.61,0.53,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,0.04,0.09,0.02,?,?,?,?,0.00,?,0.03


In [ ]:
file = open("/content/drive/MyDrive/ADM Project/communities.names")
header = []
for line in file:
    if "@attribute" in line: 
        header.append(line.split(" ")[1])

In [ ]:
df.columns = header
df.head()

,state,county,community,communityname,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,agePct65up,numbUrban,pctUrban,medIncome,pctWWage,pctWFarmSelf,pctWInvInc,pctWSocSec,pctWPubAsst,pctWRetire,medFamInc,perCapInc,whitePerCap,blackPerCap,indianPerCap,AsianPerCap,OtherPerCap,HispPerCap,NumUnderPov,PctPopUnderPov,PctLess9thGrade,PctNotHSGrad,PctBSorMore,PctUnemployed,PctEmploy,PctEmplManu,...,RentMedian,RentHighQ,MedRent,MedRentPctHousInc,MedOwnCostPctInc,MedOwnCostPctIncNoMtg,NumInShelters,NumStreet,PctForeignBorn,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,LemasSwornFT,LemasSwFTPerPop,LemasSwFTFieldOps,LemasSwFTFieldPerPop,LemasTotalReq,LemasTotReqPerPop,PolicReqPerOffic,PolicPerPop,RacialMatchCommPol,PctPolicWhite,PctPolicBlack,PctPolicHisp,PctPolicAsian,PctPolicMinor,OfficAssgnDrugUnits,NumKindsDrugsSeiz,PolicAveOTWorked,LandArea,PopDens,PctUsePubTrans,PolicCars,PolicOperBudg,LemasPctPolicOnPatr,LemasGangUnitDeploy,LemasPctOfficDrugUn,PolicBudgPerPop,ViolentCrimesPerPop
0,8,?,?,Lakewoodcity,1,0.19,0.33,0.02,0.90,0.12,0.17,0.34,0.47,0.29,0.32,0.20,1.0,0.37,0.72,0.34,0.60,0.29,0.15,0.43,0.39,0.40,0.39,0.32,0.27,0.27,0.36,0.41,0.08,0.19,0.10,0.18,0.48,0.27,0.68,0.23,...,0.35,0.38,0.34,0.38,0.46,0.25,0.04,0.0,0.12,0.42,0.50,0.51,0.64,0.03,0.13,0.96,0.17,0.06,0.18,0.44,0.13,0.94,0.93,0.03,0.07,0.1,0.07,0.02,0.57,0.29,0.12,0.26,0.20,0.06,0.04,0.9,0.5,0.32,0.14,0.20
1,53,?,?,Tukwilacity,1,0.00,0.16,0.12,0.74,0.45,0.07,0.26,0.59,0.35,0.27,0.02,1.0,0.31,0.72,0.11,0.45,0.25,0.29,0.39,0.29,0.37,0.38,0.33,0.16,0.30,0.22,0.35,0.01,0.24,0.14,0.24,0.30,0.27,0.73,0.57,...,0.38,0.40,0.37,0.29,0.32,0.18,0.00,0.0,0.21,0.50,0.34,0.60,0.52,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,0.02,0.12,0.45,?,?,?,?,0.00,?,0.67
2,24,?,?,Aberdeentown,1,0.00,0.42,0.49,0.56,0.17,0.04,0.39,0.47,0.28,0.32,0.00,0.0,0.30,0.58,0.19,0.39,0.38,0.40,0.84,0.28,0.27,0.29,0.27,0.07,0.29,0.28,0.39,0.01,0.27,0.27,0.43,0.19,0.36,0.58,0.32,...,0.29,0.27,0.31,0.48,0.39,0.28,0.00,0.0,0.14,0.49,0.54,0.67,0.56,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,0.01,0.21,0.02,?,?,?,?,0.00,?,0.43
3,34,5,81440,Willingborotownship,1,0.04,0.77,1.00,0.08,0.12,0.10,0.51,0.50,0.34,0.21,0.06,1.0,0.58,0.89,0.21,0.43,0.36,0.20,0.82,0.51,0.36,0.40,0.39,0.16,0.25,0.36,0.44,0.01,0.10,0.09,0.25,0.31,0.33,0.71,0.36,...,0.70,0.77,0.89,0.63,0.51,0.47,0.00,0.0,0.19,0.30,0.73,0.64,0.65,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,0.02,0.39,0.28,?,?,?,?,0.00,?,0.12
4,42,95,6096,Bethlehemtownship,1,0.01,0.55,0.02,0.95,0.09,0.05,0.38,0.38,0.23,0.36,0.02,0.9,0.50,0.72,0.16,0.68,0.44,0.11,0.71,0.46,0.43,0.41,0.28,0.00,0.74,0.51,0.48,0.00,0.06,0.25,0.30,0.33,0.12,0.65,0.67,...,0.36,0.38,0.38,0.22,0.51,0.21,0.00,0.0,0.11,0.72,0.64,0.61,0.53,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,0.04,0.09,0.02,?,?,?,?,0.00,?,0.03


###saving the csv to drive

In [ ]:
df.to_csv('/content/drive/MyDrive/ADM Project/communities.csv')

##Data cleaning

####checking for missing values

What are the missing values per column!!

In [ ]:
missing_per_row = []
for column in df.columns:
    try:
        count = df[df[column] == "?"].shape[0]
    except Exception:
        count = 0
    missing_per_row.append((column, count))
sorted(missing_per_row, key=lambda tup: tup[1], reverse=True)

[('LemasSwornFT', 1675),
 ('LemasSwFTPerPop', 1675),
 ('LemasSwFTFieldOps', 1675),
 ('LemasSwFTFieldPerPop', 1675),
 ('LemasTotalReq', 1675),
 ('LemasTotReqPerPop', 1675),
 ('PolicReqPerOffic', 1675),
 ('PolicPerPop', 1675),
 ('RacialMatchCommPol', 1675),
 ('PctPolicWhite', 1675),
 ('PctPolicBlack', 1675),
 ('PctPolicHisp', 1675),
 ('PctPolicAsian', 1675),
 ('PctPolicMinor', 1675),
 ('OfficAssgnDrugUnits', 1675),
 ('NumKindsDrugsSeiz', 1675),
 ('PolicAveOTWorked', 1675),
 ('PolicCars', 1675),
 ('PolicOperBudg', 1675),
 ('LemasPctPolicOnPatr', 1675),
 ('LemasGangUnitDeploy', 1675),
 ('PolicBudgPerPop', 1675),
 ('community', 1177),
 ('county', 1174),
 ('OtherPerCap', 1),
 ('state', 0),
 ('communityname', 0),
 ('fold', 0),
 ('population', 0),
 ('householdsize', 0),
 ('racepctblack', 0),
 ('racePctWhite', 0),
 ('racePctAsian', 0),
 ('racePctHisp', 0),
 ('agePct12t21', 0),
 ('agePct12t29', 0),
 ('agePct16t24', 0),
 ('agePct65up', 0),
 ('numbUrban', 0),
 ('pctUrban', 0),
 ('medIncome', 0),
 

In [ ]:
df.shape

(1994, 128)

####dropping the rows with missing values

What is number of rows with missing values!

In [ ]:
missing_value_rows = []
for i in range(df.shape[0]):
    if "?" in list(df.iloc[i].values):
        missing_value_rows.append(i)

In [ ]:
print("Total rows: " + str(df.shape[0]))
print("Rows with missing values: " + str(len(missing_value_rows)))

Total rows: 1994
Rows with missing values: 1871


drop the row with missing "OtherPerCap" information, but keep the column.
Remove the columns wiht the missing per row values 1675!

In [ ]:
df = df[df["OtherPerCap"] != "?"]
columns_to_drop = [column for column, count in missing_per_row if count == 1675]
df = df.drop(columns_to_drop, axis=1)
df.shape

(1993, 106)

In [ ]:
df.sample(5)

,state,county,community,communityname,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,agePct65up,numbUrban,pctUrban,medIncome,pctWWage,pctWFarmSelf,pctWInvInc,pctWSocSec,pctWPubAsst,pctWRetire,medFamInc,perCapInc,whitePerCap,blackPerCap,indianPerCap,AsianPerCap,OtherPerCap,HispPerCap,NumUnderPov,PctPopUnderPov,PctLess9thGrade,PctNotHSGrad,PctBSorMore,PctUnemployed,PctEmploy,PctEmplManu,...,PctNotSpeakEnglWell,PctLargHouseFam,PctLargHouseOccup,PersPerOccupHous,PersPerOwnOccHous,PersPerRentOccHous,PctPersOwnOccup,PctPersDenseHous,PctHousLess3BR,MedNumBR,HousVacant,PctHousOccup,PctHousOwnOcc,PctVacantBoarded,PctVacMore6Mos,MedYrHousBuilt,PctHousNoPhone,PctWOFullPlumb,OwnOccLowQuart,OwnOccMedVal,OwnOccHiQuart,RentLowQ,RentMedian,RentHighQ,MedRent,MedRentPctHousInc,MedOwnCostPctInc,MedOwnCostPctIncNoMtg,NumInShelters,NumStreet,PctForeignBorn,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,LandArea,PopDens,PctUsePubTrans,LemasPctOfficDrugUn,ViolentCrimesPerPop
23,19,193,93926,SiouxCitycity,1,0.11,0.43,0.04,0.89,0.09,0.06,0.45,0.48,0.31,0.46,0.13,1.0,0.22,0.52,0.44,0.49,0.56,0.41,0.39,0.25,0.25,0.25,0.16,0.07,0.20,0.19,0.22,0.10,0.36,0.29,0.37,0.27,0.33,0.53,0.43,...,0.09,0.27,0.24,0.43,0.48,0.35,0.64,0.12,0.56,0.0,0.09,0.80,0.61,0.45,0.43,0.19,0.34,0.25,0.05,0.04,0.05,0.14,0.17,0.18,0.17,0.44,0.20,0.43,0.05,0.01,0.09,0.72,0.58,0.72,0.64,0.16,0.12,0.07,0.56,0.63
586,13,?,?,Albanycity,3,0.11,0.52,1.00,0.15,0.02,0.01,0.58,0.57,0.41,0.33,0.12,1.0,0.17,0.49,0.20,0.24,0.42,0.77,0.40,0.18,0.18,0.36,0.15,0.15,0.41,0.19,0.24,0.19,0.73,0.48,0.59,0.28,0.74,0.35,0.35,...,0.03,0.38,0.35,0.51,0.48,0.56,0.31,0.34,0.52,0.5,0.14,0.64,0.34,0.45,0.37,0.56,0.70,0.41,0.09,0.09,0.11,0.06,0.12,0.16,0.17,0.52,0.28,0.42,0.02,0.00,0.02,0.80,0.50,0.69,0.71,0.16,0.12,0.10,1.00,0.56
1442,53,?,?,Redmondcity,8,0.04,0.37,0.03,0.87,0.39,0.04,0.36,0.51,0.29,0.18,0.06,1.0,0.51,0.84,0.18,0.61,0.16,0.08,0.24,0.54,0.51,0.50,0.65,0.24,0.39,0.27,0.46,0.01,0.08,0.06,0.07,0.67,0.12,0.81,0.41,...,0.11,0.17,0.15,0.40,0.51,0.29,0.54,0.10,0.52,0.5,0.04,0.79,0.48,0.03,0.03,0.83,0.04,0.09,0.43,0.43,0.42,0.62,0.57,0.61,0.58,0.40,0.41,0.25,0.00,0.00,0.28,0.36,0.29,0.45,0.33,0.04,0.21,0.27,0.00,0.06
1946,6,?,?,Westminstercity,10,0.11,0.68,0.02,0.54,1.00,0.35,0.45,0.58,0.40,0.29,0.12,1.0,0.49,0.67,0.16,0.45,0.34,0.48,0.48,0.46,0.36,0.42,0.35,0.27,0.24,0.28,0.33,0.08,0.29,0.35,0.42,0.28,0.34,0.59,0.57,...,0.77,0.76,0.73,0.74,0.62,0.90,0.48,0.67,0.51,0.5,0.04,0.91,0.55,0.29,0.14,0.58,0.07,0.19,0.62,0.60,0.58,0.76,0.69,0.84,0.67,0.77,0.54,0.18,0.00,0.00,0.87,0.36,0.48,0.63,0.62,0.03,0.65,0.11,0.00,0.20
397,42,29,60120,Phoenixvilleborough,2,0.01,0.31,0.13,0.88,0.07,0.02,0.24,0.50,0.25,0.48,0.02,1.0,0.33,0.60,0.08,0.50,0.50,0.28,0.55,0.37,0.34,0.34,0.31,0.00,0.11,0.22,0.24,0.01,0.13,0.32,0.44,0.29,0.19,0.60,0.54,...,0.07,0.20,0.17,0.33,0.43,0.24,0.53,0.06,0.48,0.5,0.02,0.80,0.48,0.22,0.43,0.27,0.09,0.08,0.22,0.20,0.20,0.45,0.43,0.46,0.44,0.45,0.54,0.58,0.00,0.00,0.10,0.82,0.61,0.60,0.73,0.01,0.35,0.10,0.00,0.09


####Compare between cut and qcut

In [ ]:
df['householdsize'].describe()

count    1993.000000
mean        0.463437
std         0.163747
min         0.000000
25%         0.350000
50%         0.440000
75%         0.540000
max         1.000000
Name: householdsize, dtype: float64

In [ ]:
pd.qcut(df['householdsize'], q=4).value_counts()

(0.35, 0.44]      528
(-0.001, 0.35]    509
(0.54, 1.0]       490
(0.44, 0.54]      466
Name: householdsize, dtype: int64

In [ ]:
pd.cut(df['householdsize'], bins=4).value_counts()

(0.25, 0.5]       1221
(0.5, 0.75]        545
(0.75, 1.0]        118
(-0.001, 0.25]     109
Name: householdsize, dtype: int64

*   It must not be necessary to have equal number of participants in each bin as long as bins make sense.

*   cut provides the same, so going along with the pd.cut.



In [ ]:
pd.cut(df['householdsize'], bins=np.linspace(df['householdsize'].min(), df['householdsize'].max(), 5)).value_counts()

(0.25, 0.5]    1221
(0.5, 0.75]     545
(0.75, 1.0]     118
(0.0, 0.25]     107
Name: householdsize, dtype: int64

last one misses some of the values, so not usable.

Cols from GA

In [ ]:
excluded_cols = ['householdsize',
 'racePctAsian',
 'agePct12t29',
 'agePct65up',
 'numbUrban',
 'medIncome',
 'pctWWage',
 'pctWFarmSelf',
 'pctWInvInc',
 'pctWSocSec',
 'pctWPubAsst',
 'pctWRetire',
 'whitePerCap',
 'indianPerCap',
 'AsianPerCap',
 'NumUnderPov',
 'PctPopUnderPov',
 'PctLess9thGrade',
 'PctNotHSGrad',
 'PctUnemployed',
 'PctEmploy',
 'PctOccupMgmtProf',
 'MalePctDivorce',
 'MalePctNevMarr',
 'FemalePctDiv',
 'TotalPctDiv',
 'PersPerFam',
 'PctFam2Par',
 'PctTeen2Par',
 'PctWorkMomYoungKids',
 'PctWorkMom',
 'NumImmig',
 'PctImmigRec5',
 'PctImmigRec8',
 'PctImmigRec10',
 'PctRecentImmig',
 'PctRecImmig8',
 'PctNotSpeakEnglWell',
 'PctLargHouseFam',
 'PctLargHouseOccup',
 'PersPerOwnOccHous',
 'PctPersDenseHous',
 'MedNumBR',
 'PctHousOccup',
 'PctHousOwnOcc',
 'PctVacantBoarded',
 'PctVacMore6Mos',
 'PctWOFullPlumb',
 'OwnOccHiQuart',
 'RentLowQ',
 'RentHighQ',
 'MedRent',
 'NumInShelters',
 'PctForeignBorn',
 'PctBornSameState',
 'PctSameState85',
 'LandArea',
 'PopDens',
 'LemasPctOfficDrugUn']

In [ ]:
included_cols = ['population',
 'racepctblack',
 'racePctWhite',
 'racePctHisp',
 'agePct12t21',
 'agePct16t24',
 'pctUrban',
 'medFamInc',
 'perCapInc',
 'blackPerCap',
 'OtherPerCap',
 'HispPerCap',
 'PctBSorMore',
 'PctEmplManu',
 'PctEmplProfServ',
 'PctOccupManu',
 'PctKids2Par',
 'PctYoungKids2Par',
 'NumIlleg',
 'PctIlleg',
 'PctImmigRecent',
 'PctRecImmig5',
 'PctRecImmig10',
 'PctSpeakEnglOnly',
 'PersPerOccupHous',
 'PersPerRentOccHous',
 'PctPersOwnOccup',
 'PctHousLess3BR',
 'HousVacant',
 'MedYrHousBuilt',
 'PctHousNoPhone',
 'OwnOccLowQuart',
 'OwnOccMedVal',
 'RentMedian',
 'MedRentPctHousInc',
 'MedOwnCostPctInc',
 'MedOwnCostPctIncNoMtg',
 'NumStreet',
 'PctSameHouse85',
 'PctSameCity85',
 'PctUsePubTrans']

###Dropping the excluded columns

In [ ]:
for name in excluded_cols:
  df = df.drop(name, axis=1)

In [ ]:
df.shape

(1993, 47)

###Using cut to create bins 
For simplicity of the further process in fuzzyfication, the bins number is kept 4.

In [ ]:
for name in included_cols:
  print(name)
  if df[name].dtype == str or df[name].dtype == object:
    print('skipped ^')
    continue
  df[name] = pd.cut(df[name], bins=4)

population
racepctblack
racePctWhite
racePctHisp
agePct12t21
agePct16t24
pctUrban
medFamInc
perCapInc
blackPerCap
OtherPerCap
skipped ^
HispPerCap
PctBSorMore
PctEmplManu
PctEmplProfServ
PctOccupManu
PctKids2Par
PctYoungKids2Par
NumIlleg
PctIlleg
PctImmigRecent
PctRecImmig5
PctRecImmig10
PctSpeakEnglOnly
PersPerOccupHous
PersPerRentOccHous
PctPersOwnOccup
PctHousLess3BR
HousVacant
MedYrHousBuilt
PctHousNoPhone
OwnOccLowQuart
OwnOccMedVal
RentMedian
MedRentPctHousInc
MedOwnCostPctInc
MedOwnCostPctIncNoMtg
NumStreet
PctSameHouse85
PctSameCity85
PctUsePubTrans


Making the final col ViolentCrimesPerPop too descritized in 3 bins as per output given in the form.

In [ ]:
df['ViolentCrimesPerPop'] = pd.cut(df['ViolentCrimesPerPop'], bins = 3)

In [ ]:
df['OtherPerCap'] = df['OtherPerCap'].astype(float)

In [ ]:
df['OtherPerCap'] = pd.cut(df['OtherPerCap'], bins=4)

###Checking final data

In [ ]:
df.sample(10)

,state,county,community,communityname,fold,population,racepctblack,racePctWhite,racePctHisp,agePct12t21,agePct16t24,pctUrban,medFamInc,perCapInc,blackPerCap,OtherPerCap,HispPerCap,PctBSorMore,PctEmplManu,PctEmplProfServ,PctOccupManu,PctKids2Par,PctYoungKids2Par,NumIlleg,PctIlleg,PctImmigRecent,PctRecImmig5,PctRecImmig10,PctSpeakEnglOnly,PersPerOccupHous,PersPerRentOccHous,PctPersOwnOccup,PctHousLess3BR,HousVacant,MedYrHousBuilt,PctHousNoPhone,OwnOccLowQuart,OwnOccMedVal,RentMedian,MedRentPctHousInc,MedOwnCostPctInc,MedOwnCostPctIncNoMtg,NumStreet,PctSameHouse85,PctSameCity85,PctUsePubTrans,ViolentCrimesPerPop
549,36,1,16749,Cohoescity,3,"(-0.001, 0.25]","(-0.001, 0.25]","(0.75, 1.0]","(-0.001, 0.25]","(0.25, 0.5]","(0.25, 0.5]","(0.75, 1.0]","(0.25, 0.5]","(-0.001, 0.25]","(0.25, 0.5]","(0.25, 0.5]","(0.25, 0.5]","(-0.001, 0.25]","(0.25, 0.5]","(0.25, 0.5]","(0.25, 0.5]","(0.25, 0.5]","(0.25, 0.5]","(-0.001, 0.25]","(0.25, 0.5]","(-0.001, 0.25]","(-0.001, 0.25]","(-0.001, 0.25]","(0.75, 1.0]","(0.25, 0.5]","(0.25, 0.5]","(0.25, 0.5]","(0.25, 0.5]","(-0.001, 0.25]","(-0.001, 0.25]","(0.25, 0.5]","(-0.001, 0.25]","(-0.001, 0.25]","(-0.001, 0.25]","(0.25, 0.5]","(0.25, 0.5]","(0.5, 0.75]","(-0.001, 0.25]","(0.5, 0.75]","(0.5, 0.75]","(-0.001, 0.25]","(-0.001, 0.333]"
1034,48,?,?,Gatesvillecity,6,"(-0.001, 0.25]","(0.25, 0.5]","(0.25, 0.5]","(-0.001, 0.25]","(0.25, 0.5]","(0.5, 0.75]","(-0.001, 0.25]","(-0.001, 0.25]","(-0.001, 0.25]","(-0.001, 0.25]","(-0.001, 0.25]","(-0.001, 0.25]","(-0.001, 0.25]","(0.25, 0.5]","(0.25, 0.5]","(0.25, 0.5]","(0.5, 0.75]","(0.5, 0.75]","(-0.001, 0.25]","(0.75, 1.0]","(0.25, 0.5]","(-0.001, 0.25]","(-0.001, 0.25]","(0.75, 1.0]","(0.25, 0.5]","(0.25, 0.5]","(0.5, 0.75]","(0.5, 0.75]","(-0.001, 0.25]","(0.25, 0.5]","(0.5, 0.75]","(-0.001, 0.25]","(-0.001, 0.25]","(-0.001, 0.25]","(0.25, 0.5]","(0.25, 0.5]","(0.25, 0.5]","(-0.001, 0.25]","(-0.001, 0.25]","(-0.001, 0.25]","(-0.001, 0.25]","(-0.001, 0.333]"
1890,5,?,?,Bentonvillecity,10,"(-0.001, 0.25]","(-0.001, 0.25]","(0.75, 1.0]","(-0.001, 0.25]","(0.25, 0.5]","(0.25, 0.5]","(-0.001, 0.25]","(-0.001, 0.25]","(-0.001, 0.25]","(0.25, 0.5]","(-0.001, 0.25]","(0.25, 0.5]","(-0.001, 0.25]","(0.25, 0.5]","(-0.001, 0.25]","(0.5, 0.75]","(0.5, 0.75]","(0.5, 0.75]","(-0.001, 0.25]","(-0.001, 0.25]","(-0.001, 0.25]","(-0.001, 0.25]","(-0.001, 0.25]","(0.75, 1.0]","(0.25, 0.5]","(0.25, 0.5]","(0.5, 0.75]","(0.25, 0.5]","(-0.001, 0.25]","(0.5, 0.75]","(0.5, 0.75]","(-0.001, 0.25]","(-0.001, 0.25]","(-0.001, 0.25]","(0.25, 0.5]","(0.25, 0.5]","(0.25, 0.5]","(-0.001, 0.25]","(0.25, 0.5]","(0.5, 0.75]","(-0.001, 0.25]","(-0.001, 0.333]"
1695,48,?,?,Arlingtoncity,9,"(0.25, 0.5]","(-0.001, 0.25]","(0.5, 0.75]","(-0.001, 0.25]","(0.25, 0.5]","(0.25, 0.5]","(0.75, 1.0]","(0.25, 0.5]","(0.25, 0.5]","(0.25, 0.5]","(-0.001, 0.25]","(0.25, 0.5]","(0.25, 0.5]","(0.25, 0.5]","(0.25, 0.5]","(0.25, 0.5]","(0.5, 0.75]","(0.75, 1.0]","(-0.001, 0.25]","(-0.001, 0.25]","(0.25, 0.5]","(-0.001, 0.25]","(0.25, 0.5]","(0.75, 1.0]","(0.25, 0.5]","(0.25, 0.5]","(0.25, 0.5]","(0.5, 0.75]","(0.5, 0.75]","(0.75, 1.0]","(0.25, 0.5]","(-0.001, 0.25]","(-0.001, 0.25]","(0.25, 0.5]","(0.25, 0.5]","(0.25, 0.5]","(-0.001, 0.25]","(-0.001, 0.25]","(0.25, 0.5]","(0.25, 0.5]","(-0.001, 0.25]","(-0.001, 0.333]"
516,34,13,4695,Bellevilletownship,3,"(-0.001, 0.25]","(-0.001, 0.25]","(0.75, 1.0]","(-0.001, 0.25]","(0.25, 0.5]","(0.25, 0.5]","(0.75, 1.0]","(0.25, 0.5]","(0.25, 0.5]","(0.25, 0.5]","(0.25, 0.5]","(0.5, 0.75]","(0.25, 0.5]","(0.25, 0.5]","(0.25, 0.5]","(0.25, 0.5]","(0.5, 0.75]","(0.5, 0.75]","(-0.001, 0.25]","(-0.001, 0.25]","(-0.001, 0.25]","(0.25, 0.5]","(0.25, 0.5]","(0.5, 0.75]","(0.25, 0.5]","(0.25, 0.5]","(0.25, 0.5]","(0.5, 0.75]","(-0.001, 0.25]","(-0.001, 0.25]","(-0.001, 0.25]","(0.25, 0.5]","(0.25, 0.5]","(0.5, 0.75]","(0.25, 0.5]","(0.5, 0.75]","(0.75, 1.0]","(-0.001, 0.25]","(0.75, 1.0]","(0.75, 1.0]","(0.25, 0.5]","(-0.001, 0.333]"
590,40,?,?,Okmulgeecity,3,"(-0.001, 0.25]

###Saving the final data

In [ ]:
df.to_csv('/content/drive/MyDrive/ADM Project/descritized_crimedata.csv')

##Removing the informational part of data such as community name and country code and fold

In [ ]:
df = df.drop(["state", "county", "community", "communityname", "fold"], axis=1)
df.shape

(1993, 42)

In [ ]:
df.sample(5)

,population,racepctblack,racePctWhite,racePctHisp,agePct12t21,agePct16t24,pctUrban,medFamInc,perCapInc,blackPerCap,OtherPerCap,HispPerCap,PctBSorMore,PctEmplManu,PctEmplProfServ,PctOccupManu,PctKids2Par,PctYoungKids2Par,NumIlleg,PctIlleg,PctImmigRecent,PctRecImmig5,PctRecImmig10,PctSpeakEnglOnly,PersPerOccupHous,PersPerRentOccHous,PctPersOwnOccup,PctHousLess3BR,HousVacant,MedYrHousBuilt,PctHousNoPhone,OwnOccLowQuart,OwnOccMedVal,RentMedian,MedRentPctHousInc,MedOwnCostPctInc,MedOwnCostPctIncNoMtg,NumStreet,PctSameHouse85,PctSameCity85,PctUsePubTrans,ViolentCrimesPerPop
1826,"(-0.001, 0.25]","(-0.001, 0.25]","(0.75, 1.0]","(-0.001, 0.25]","(0.25, 0.5]","(-0.001, 0.25]","(0.75, 1.0]","(0.5, 0.75]","(0.25, 0.5]","(0.25, 0.5]","(0.25, 0.5]","(0.5, 0.75]","(0.25, 0.5]","(0.5, 0.75]","(0.25, 0.5]","(0.25, 0.5]","(0.75, 1.0]","(0.75, 1.0]","(-0.001, 0.25]","(-0.001, 0.25]","(0.25, 0.5]","(-0.001, 0.25]","(-0.001, 0.25]","(0.75, 1.0]","(0.25, 0.5]","(-0.001, 0.25]","(0.75, 1.0]","(0.25, 0.5]","(-0.001, 0.25]","(0.25, 0.5]","(-0.001, 0.25]","(0.25, 0.5]","(0.25, 0.5]","(0.5, 0.75]","(0.5, 0.75]","(0.5, 0.75]","(0.5, 0.75]","(-0.001, 0.25]","(0.75, 1.0]","(0.75, 1.0]","(-0.001, 0.25]","(-0.001, 0.333]"
159,"(-0.001, 0.25]","(-0.001, 0.25]","(0.75, 1.0]","(-0.001, 0.25]","(0.25, 0.5]","(-0.001, 0.25]","(-0.001, 0.25]","(0.25, 0.5]","(-0.001, 0.25]","(0.25, 0.5]","(0.25, 0.5]","(0.25, 0.5]","(-0.001, 0.25]","(0.25, 0.5]","(0.25, 0.5]","(0.25, 0.5]","(0.75, 1.0]","(0.75, 1.0]","(-0.001, 0.25]","(-0.001, 0.25]","(-0.001, 0.25]","(-0.001, 0.25]","(-0.001, 0.25]","(0.75, 1.0]","(0.5, 0.75]","(0.25, 0.5]","(0.5, 0.75]","(0.25, 0.5]","(-0.001, 0.25]","(0.75, 1.0]","(-0.001, 0.25]","(-0.001, 0.25]","(-0.001, 0.25]","(0.25, 0.5]","(0.5, 0.75]","(0.25, 0.5]","(0.25, 0.5]","(-0.001, 0.25]","(0.25, 0.5]","(0.5, 0.75]","(-0.001, 0.25]","(-0.001, 0.333]"
1675,"(-0.001, 0.25]","(-0.001, 0.25]","(0.75, 1.0]","(-0.001, 0.25]","(0.25, 0.5]","(0.25, 0.5]","(0.75, 1.0]","(0.5, 0.75]","(0.25, 0.5]","(0.75, 1.0]","(0.5, 0.75]","(0.25, 0.5]","(0.25, 0.5]","(0.25, 0.5]","(0.25, 0.5]","(0.25, 0.5]","(0.75, 1.0]","(0.75, 1.0]","(-0.001, 0.25]","(-0.001, 0.25]","(-0.001, 0.25]","(-0.001, 0.25]","(-0.001, 0.25]","(0.75, 1.0]","(0.75, 1.0]","(0.5, 0.75]","(0.75, 1.0]","(-0.001, 0.25]","(-0.001, 0.25]","(0.25, 0.5]","(-0.001, 0.25]","(0.25, 0.5]","(0.25, 0.5]","(0.5, 0.75]","(0.5, 0.75]","(0.5, 0.75]","(0.25, 0.5]","(-0.001, 0.25]","(0.75, 1.0]","(0.75, 1.0]","(-0.001, 0.25]","(-0.001, 0.333]"
263,"(-0.001, 0.25]","(-0.001, 0.25]","(0.75, 1.0]","(-0.001, 0.25]","(0.25, 0.5]","(0.25, 0.5]","(0.75, 1.0]","(0.5, 0.75]","(0.5, 0.75]","(-0.001, 0.25]","(-0.001, 0.25]","(0.5, 0.75]","(0.5, 0.75]","(0.25, 0.5]","(0.5, 0.75]","(-0.001, 0.25]","(0.75, 1.0]","(0.75, 1.0]","(-0.001, 0.25]","(-0.001, 0.25]","(-0.001, 0.25]","(-0.001, 0.25]","(-0.001, 0.25]","(0.75, 1.0]","(0.5, 0.75]","(0.25, 0.5]","(0.75, 1.0]","(-0.001, 0.25]","(-0.001, 0.25]","(0.5, 0.75]","(-0.001, 0.25]","(0.5, 0.75]","(0.5, 0.75]","(0.5, 0.75]","(0.5, 0.75]","(0.5, 0.75]","(0.25, 0.5]","(-0.001, 0.25]","(0.5, 0.75]","(0.5, 0.75]","(-0.001, 0.25]","(-0.001, 0.333]"
62,"(-0.001, 0.25]","(-0.001, 0.25]","(0.75, 1.0]","(-0.001, 0.25]","(0.5, 0.75]","(0.25, 0.5]","(-0.001, 0.25]","(-0.001, 0.25]","(-0.001, 0.25]","(-0.001, 0.25]","(0.25, 0.5]","(0.25, 0.5]","(0.25, 0.5]","(0.25, 0.5]","(0.5, 0.75]","(0.5, 0.75]","(0.5, 0.75]","(0.5, 0.75]","(-0.001, 0.25]","(-0.001, 0.25]","(0.75, 1.0]","(-0.001, 0.25]","(-0.001, 0.25]","(0.75, 1.0]","(0.25, 0.5]","(0.25, 0.5]","(0.25, 0.5]","(0.5, 0.75]","(-0.001, 0.25]","(0.25, 0.5]","(0.25, 0.5]","(-0.001, 0.25]","(-0.001, 0.25]","(-0.001, 0.25]","(0.25, 0.5]","(-0.001, 0.25]","(0.25, 0.5]","(-0.001, 0.25]","(0.25, 0.5]","(0.25, 0.5]","(-0.001, 0.25]","(-0.001, 0.333]"


###Saving the data

In [ ]:
df.to_csv('/content/drive/MyDrive/ADM Project/descritized_crimedata_without_info_cols.csv')

#Encoded descritized values


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/ADM Project/communities.data", header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127
0,8,?,?,Lakewoodcity,1,0.19,0.33,0.02,0.90,0.12,0.17,0.34,0.47,0.29,0.32,0.20,1.0,0.37,0.72,0.34,0.60,0.29,0.15,0.43,0.39,0.40,0.39,0.32,0.27,0.27,0.36,0.41,0.08,0.19,0.10,0.18,0.48,0.27,0.68,0.23,...,0.35,0.38,0.34,0.38,0.46,0.25,0.04,0.0,0.12,0.42,0.50,0.51,0.64,0.03,0.13,0.96,0.17,0.06,0.18,0.44,0.13,0.94,0.93,0.03,0.07,0.1,0.07,0.02,0.57,0.29,0.12,0.26,0.20,0.06,0.04,0.9,0.5,0.32,0.14,0.20
1,53,?,?,Tukwilacity,1,0.00,0.16,0.12,0.74,0.45,0.07,0.26,0.59,0.35,0.27,0.02,1.0,0.31,0.72,0.11,0.45,0.25,0.29,0.39,0.29,0.37,0.38,0.33,0.16,0.30,0.22,0.35,0.01,0.24,0.14,0.24,0.30,0.27,0.73,0.57,...,0.38,0.40,0.37,0.29,0.32,0.18,0.00,0.0,0.21,0.50,0.34,0.60,0.52,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,0.02,0.12,0.45,?,?,?,?,0.00,?,0.67
2,24,?,?,Aberdeentown,1,0.00,0.42,0.49,0.56,0.17,0.04,0.39,0.47,0.28,0.32,0.00,0.0,0.30,0.58,0.19,0.39,0.38,0.40,0.84,0.28,0.27,0.29,0.27,0.07,0.29,0.28,0.39,0.01,0.27,0.27,0.43,0.19,0.36,0.58,0.32,...,0.29,0.27,0.31,0.48,0.39,0.28,0.00,0.0,0.14,0.49,0.54,0.67,0.56,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,0.01,0.21,0.02,?,?,?,?,0.00,?,0.43
3,34,5,81440,Willingborotownship,1,0.04,0.77,1.00,0.08,0.12,0.10,0.51,0.50,0.34,0.21,0.06,1.0,0.58,0.89,0.21,0.43,0.36,0.20,0.82,0.51,0.36,0.40,0.39,0.16,0.25,0.36,0.44,0.01,0.10,0.09,0.25,0.31,0.33,0.71,0.36,...,0.70,0.77,0.89,0.63,0.51,0.47,0.00,0.0,0.19,0.30,0.73,0.64,0.65,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,0.02,0.39,0.28,?,?,?,?,0.00,?,0.12
4,42,95,6096,Bethlehemtownship,1,0.01,0.55,0.02,0.95,0.09,0.05,0.38,0.38,0.23,0.36,0.02,0.9,0.50,0.72,0.16,0.68,0.44,0.11,0.71,0.46,0.43,0.41,0.28,0.00,0.74,0.51,0.48,0.00,0.06,0.25,0.30,0.33,0.12,0.65,0.67,...,0.36,0.38,0.38,0.22,0.51,0.21,0.00,0.0,0.11,0.72,0.64,0.61,0.53,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,0.04,0.09,0.02,?,?,?,?,0.00,?,0.03


In [ ]:
file = open("/content/drive/MyDrive/ADM Project/communities.names")
header = []
for line in file:
    if "@attribute" in line: 
        header.append(line.split(" ")[1])

In [ ]:
df.columns = header
df.head()

,state,county,community,communityname,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,agePct65up,numbUrban,pctUrban,medIncome,pctWWage,pctWFarmSelf,pctWInvInc,pctWSocSec,pctWPubAsst,pctWRetire,medFamInc,perCapInc,whitePerCap,blackPerCap,indianPerCap,AsianPerCap,OtherPerCap,HispPerCap,NumUnderPov,PctPopUnderPov,PctLess9thGrade,PctNotHSGrad,PctBSorMore,PctUnemployed,PctEmploy,PctEmplManu,...,RentMedian,RentHighQ,MedRent,MedRentPctHousInc,MedOwnCostPctInc,MedOwnCostPctIncNoMtg,NumInShelters,NumStreet,PctForeignBorn,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,LemasSwornFT,LemasSwFTPerPop,LemasSwFTFieldOps,LemasSwFTFieldPerPop,LemasTotalReq,LemasTotReqPerPop,PolicReqPerOffic,PolicPerPop,RacialMatchCommPol,PctPolicWhite,PctPolicBlack,PctPolicHisp,PctPolicAsian,PctPolicMinor,OfficAssgnDrugUnits,NumKindsDrugsSeiz,PolicAveOTWorked,LandArea,PopDens,PctUsePubTrans,PolicCars,PolicOperBudg,LemasPctPolicOnPatr,LemasGangUnitDeploy,LemasPctOfficDrugUn,PolicBudgPerPop,ViolentCrimesPerPop
0,8,?,?,Lakewoodcity,1,0.19,0.33,0.02,0.90,0.12,0.17,0.34,0.47,0.29,0.32,0.20,1.0,0.37,0.72,0.34,0.60,0.29,0.15,0.43,0.39,0.40,0.39,0.32,0.27,0.27,0.36,0.41,0.08,0.19,0.10,0.18,0.48,0.27,0.68,0.23,...,0.35,0.38,0.34,0.38,0.46,0.25,0.04,0.0,0.12,0.42,0.50,0.51,0.64,0.03,0.13,0.96,0.17,0.06,0.18,0.44,0.13,0.94,0.93,0.03,0.07,0.1,0.07,0.02,0.57,0.29,0.12,0.26,0.20,0.06,0.04,0.9,0.5,0.32,0.14,0.20
1,53,?,?,Tukwilacity,1,0.00,0.16,0.12,0.74,0.45,0.07,0.26,0.59,0.35,0.27,0.02,1.0,0.31,0.72,0.11,0.45,0.25,0.29,0.39,0.29,0.37,0.38,0.33,0.16,0.30,0.22,0.35,0.01,0.24,0.14,0.24,0.30,0.27,0.73,0.57,...,0.38,0.40,0.37,0.29,0.32,0.18,0.00,0.0,0.21,0.50,0.34,0.60,0.52,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,0.02,0.12,0.45,?,?,?,?,0.00,?,0.67
2,24,?,?,Aberdeentown,1,0.00,0.42,0.49,0.56,0.17,0.04,0.39,0.47,0.28,0.32,0.00,0.0,0.30,0.58,0.19,0.39,0.38,0.40,0.84,0.28,0.27,0.29,0.27,0.07,0.29,0.28,0.39,0.01,0.27,0.27,0.43,0.19,0.36,0.58,0.32,...,0.29,0.27,0.31,0.48,0.39,0.28,0.00,0.0,0.14,0.49,0.54,0.67,0.56,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,0.01,0.21,0.02,?,?,?,?,0.00,?,0.43
3,34,5,81440,Willingborotownship,1,0.04,0.77,1.00,0.08,0.12,0.10,0.51,0.50,0.34,0.21,0.06,1.0,0.58,0.89,0.21,0.43,0.36,0.20,0.82,0.51,0.36,0.40,0.39,0.16,0.25,0.36,0.44,0.01,0.10,0.09,0.25,0.31,0.33,0.71,0.36,...,0.70,0.77,0.89,0.63,0.51,0.47,0.00,0.0,0.19,0.30,0.73,0.64,0.65,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,0.02,0.39,0.28,?,?,?,?,0.00,?,0.12
4,42,95,6096,Bethlehemtownship,1,0.01,0.55,0.02,0.95,0.09,0.05,0.38,0.38,0.23,0.36,0.02,0.9,0.50,0.72,0.16,0.68,0.44,0.11,0.71,0.46,0.43,0.41,0.28,0.00,0.74,0.51,0.48,0.00,0.06,0.25,0.30,0.33,0.12,0.65,0.67,...,0.36,0.38,0.38,0.22,0.51,0.21,0.00,0.0,0.11,0.72,0.64,0.61,0.53,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,0.04,0.09,0.02,?,?,?,?,0.00,?,0.03


##Data cleaning

####checking for missing values

What are the missing values per column!!

In [ ]:
missing_per_row = []
for column in df.columns:
    try:
        count = df[df[column] == "?"].shape[0]
    except Exception:
        count = 0
    missing_per_row.append((column, count))
sorted(missing_per_row, key=lambda tup: tup[1], reverse=True)

[('LemasSwornFT', 1675),
 ('LemasSwFTPerPop', 1675),
 ('LemasSwFTFieldOps', 1675),
 ('LemasSwFTFieldPerPop', 1675),
 ('LemasTotalReq', 1675),
 ('LemasTotReqPerPop', 1675),
 ('PolicReqPerOffic', 1675),
 ('PolicPerPop', 1675),
 ('RacialMatchCommPol', 1675),
 ('PctPolicWhite', 1675),
 ('PctPolicBlack', 1675),
 ('PctPolicHisp', 1675),
 ('PctPolicAsian', 1675),
 ('PctPolicMinor', 1675),
 ('OfficAssgnDrugUnits', 1675),
 ('NumKindsDrugsSeiz', 1675),
 ('PolicAveOTWorked', 1675),
 ('PolicCars', 1675),
 ('PolicOperBudg', 1675),
 ('LemasPctPolicOnPatr', 1675),
 ('LemasGangUnitDeploy', 1675),
 ('PolicBudgPerPop', 1675),
 ('community', 1177),
 ('county', 1174),
 ('OtherPerCap', 1),
 ('state', 0),
 ('communityname', 0),
 ('fold', 0),
 ('population', 0),
 ('householdsize', 0),
 ('racepctblack', 0),
 ('racePctWhite', 0),
 ('racePctAsian', 0),
 ('racePctHisp', 0),
 ('agePct12t21', 0),
 ('agePct12t29', 0),
 ('agePct16t24', 0),
 ('agePct65up', 0),
 ('numbUrban', 0),
 ('pctUrban', 0),
 ('medIncome', 0),
 

In [ ]:
df.shape

(1994, 128)

####dropping the rows with missing values

What is number of rows with missing values!

In [ ]:
missing_value_rows = []
for i in range(df.shape[0]):
    if "?" in list(df.iloc[i].values):
        missing_value_rows.append(i)

In [ ]:
print("Total rows: " + str(df.shape[0]))
print("Rows with missing values: " + str(len(missing_value_rows)))

Total rows: 1994
Rows with missing values: 1871


drop the row with missing "OtherPerCap" information, but keep the column.
Remove the columns wiht the missing per row values 1675!

In [ ]:
df = df[df["OtherPerCap"] != "?"]
columns_to_drop = [column for column, count in missing_per_row if count == 1675]
df = df.drop(columns_to_drop, axis=1)
df.shape

(1993, 106)

In [ ]:
df.sample(5)

,state,county,community,communityname,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,agePct65up,numbUrban,pctUrban,medIncome,pctWWage,pctWFarmSelf,pctWInvInc,pctWSocSec,pctWPubAsst,pctWRetire,medFamInc,perCapInc,whitePerCap,blackPerCap,indianPerCap,AsianPerCap,OtherPerCap,HispPerCap,NumUnderPov,PctPopUnderPov,PctLess9thGrade,PctNotHSGrad,PctBSorMore,PctUnemployed,PctEmploy,PctEmplManu,...,PctNotSpeakEnglWell,PctLargHouseFam,PctLargHouseOccup,PersPerOccupHous,PersPerOwnOccHous,PersPerRentOccHous,PctPersOwnOccup,PctPersDenseHous,PctHousLess3BR,MedNumBR,HousVacant,PctHousOccup,PctHousOwnOcc,PctVacantBoarded,PctVacMore6Mos,MedYrHousBuilt,PctHousNoPhone,PctWOFullPlumb,OwnOccLowQuart,OwnOccMedVal,OwnOccHiQuart,RentLowQ,RentMedian,RentHighQ,MedRent,MedRentPctHousInc,MedOwnCostPctInc,MedOwnCostPctIncNoMtg,NumInShelters,NumStreet,PctForeignBorn,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,LandArea,PopDens,PctUsePubTrans,LemasPctOfficDrugUn,ViolentCrimesPerPop
525,35,?,?,Albuquerquecity,3,0.60,0.36,0.06,0.67,0.10,0.64,0.39,0.51,0.32,0.35,0.61,1.0,0.26,0.61,0.26,0.50,0.37,0.25,0.51,0.29,0.30,0.34,0.26,0.14,0.27,0.27,0.33,0.48,0.36,0.20,0.26,0.46,0.39,0.55,0.19,...,0.17,0.22,0.19,0.38,0.46,0.31,0.51,0.22,0.55,0.0,0.70,0.69,0.48,0.16,0.22,0.67,0.39,0.16,0.18,0.18,0.18,0.27,0.28,0.32,0.26,0.55,0.51,0.29,0.15,0.26,0.16,0.42,0.43,0.59,0.48,0.38,0.24,0.11,0.77,0.66
1980,25,5,69170,Tauntoncity,10,0.06,0.43,0.04,0.93,0.03,0.09,0.36,0.51,0.31,0.45,0.08,1.0,0.34,0.53,0.12,0.39,0.50,0.49,0.51,0.36,0.29,0.28,0.23,0.06,0.24,0.36,0.32,0.04,0.21,0.64,0.58,0.18,0.58,0.53,0.58,...,0.32,0.21,0.20,0.45,0.53,0.37,0.52,0.10,0.55,0.0,0.07,0.72,0.48,0.11,0.49,0.25,0.17,0.41,0.36,0.34,0.31,0.30,0.40,0.46,0.41,0.49,0.52,0.47,0.00,0.00,0.35,0.79,0.66,0.77,0.82,0.13,0.09,0.11,0.00,0.28
1928,29,?,?,Grandviewcity,10,0.02,0.46,0.33,0.71,0.06,0.03,0.50,0.57,0.38,0.21,0.04,1.0,0.35,0.80,0.42,0.43,0.26,0.18,0.44,0.36,0.31,0.31,0.33,0.28,0.27,0.25,0.36,0.01,0.16,0.13,0.23,0.27,0.28,0.74,0.37,...,0.07,0.25,0.24,0.49,0.57,0.36,0.59,0.11,0.43,0.5,0.05,0.58,0.54,0.12,0.26,0.63,0.19,0.10,0.12,0.11,0.10,0.26,0.25,0.26,0.25,0.39,0.20,0.19,0.00,0.00,0.05,0.57,0.51,0.63,0.60,0.04,0.14,0.04,0.00,0.17
1425,55,105,37825,Janesvillecity,8,0.07,0.39,0.01,0.98,0.05,0.02,0.37,0.47,0.28,0.37,0.08,1.0,0.33,0.61,0.23,0.60,0.45,0.34,0.48,0.34,0.32,0.31,0.20,0.15,0.18,0.16,0.26,0.04,0.21,0.21,0.31,0.24,0.35,0.59,0.81,...,0.03,0.18,0.17,0.42,0.48,0.31,0.63,0.06,0.45,0.5,0.04,0.88,0.59,0.31,0.39,0.48,0.16,0.15,0.11,0.09,0.08,0.24,0.25,0.26,0.27,0.36,0.18,0.36,0.02,0.00,0.05,0.82,0.58,0.78,0.77,0.07,0.18,0.06,0.00,0.08
1206,53,?,?,Puyallupcity,7,0.02,0.45,0.01,0.92,0.17,0.04,0.40,0.45,0.26,0.38,0.04,1.0,0.35,0.57,0.21,0.52,0.46,0.33,0.51,0.37,0.29,0.28,0.28,0.14,0.24,0.32,0.24,0.01,0.17,0.21,0.27,0.27,0.33,0.49,0.53,...,0.05,0.19,0.18,0.45,0.57,0.31,0.57,0.12,0.50,0.5,0.02,0.83,0.51,0.14,0.22,0.67,0.16,0.15,0.18,0.18,0.19,0.34,0.33,0.35,0.34,0.46,0.46,0.29,0.00,0.00,0.12,0.50,0.39,0.61,0.61,0.03,0.19,0.10,0.00,0.12


Cols from GA

In [ ]:
excluded_cols = ['householdsize',
 'racePctAsian',
 'agePct12t29',
 'agePct65up',
 'numbUrban',
 'medIncome',
 'pctWWage',
 'pctWFarmSelf',
 'pctWInvInc',
 'pctWSocSec',
 'pctWPubAsst',
 'pctWRetire',
 'whitePerCap',
 'indianPerCap',
 'AsianPerCap',
 'NumUnderPov',
 'PctPopUnderPov',
 'PctLess9thGrade',
 'PctNotHSGrad',
 'PctUnemployed',
 'PctEmploy',
 'PctOccupMgmtProf',
 'MalePctDivorce',
 'MalePctNevMarr',
 'FemalePctDiv',
 'TotalPctDiv',
 'PersPerFam',
 'PctFam2Par',
 'PctTeen2Par',
 'PctWorkMomYoungKids',
 'PctWorkMom',
 'NumImmig',
 'PctImmigRec5',
 'PctImmigRec8',
 'PctImmigRec10',
 'PctRecentImmig',
 'PctRecImmig8',
 'PctNotSpeakEnglWell',
 'PctLargHouseFam',
 'PctLargHouseOccup',
 'PersPerOwnOccHous',
 'PctPersDenseHous',
 'MedNumBR',
 'PctHousOccup',
 'PctHousOwnOcc',
 'PctVacantBoarded',
 'PctVacMore6Mos',
 'PctWOFullPlumb',
 'OwnOccHiQuart',
 'RentLowQ',
 'RentHighQ',
 'MedRent',
 'NumInShelters',
 'PctForeignBorn',
 'PctBornSameState',
 'PctSameState85',
 'LandArea',
 'PopDens',
 'LemasPctOfficDrugUn']

In [ ]:
included_cols = ['population',
 'racepctblack',
 'racePctWhite',
 'racePctHisp',
 'agePct12t21',
 'agePct16t24',
 'pctUrban',
 'medFamInc',
 'perCapInc',
 'blackPerCap',
 'OtherPerCap',
 'HispPerCap',
 'PctBSorMore',
 'PctEmplManu',
 'PctEmplProfServ',
 'PctOccupManu',
 'PctKids2Par',
 'PctYoungKids2Par',
 'NumIlleg',
 'PctIlleg',
 'PctImmigRecent',
 'PctRecImmig5',
 'PctRecImmig10',
 'PctSpeakEnglOnly',
 'PersPerOccupHous',
 'PersPerRentOccHous',
 'PctPersOwnOccup',
 'PctHousLess3BR',
 'HousVacant',
 'MedYrHousBuilt',
 'PctHousNoPhone',
 'OwnOccLowQuart',
 'OwnOccMedVal',
 'RentMedian',
 'MedRentPctHousInc',
 'MedOwnCostPctInc',
 'MedOwnCostPctIncNoMtg',
 'NumStreet',
 'PctSameHouse85',
 'PctSameCity85',
 'PctUsePubTrans']

###Dropping the excluded columns

In [ ]:
for name in excluded_cols:
  df = df.drop(name, axis=1)

In [ ]:
df.shape

(1993, 47)

####Cut labels

In [ ]:
cut_labels = [1,2,3,4]

In [ ]:
for name in included_cols:
  print(name)
  if df[name].dtype == str or df[name].dtype == object:
    print('skipped ^')
    continue
  df[name] = pd.cut(df[name], bins=4, labels = cut_labels)

population
racepctblack
racePctWhite
racePctHisp
agePct12t21
agePct16t24
pctUrban
medFamInc
perCapInc
blackPerCap
OtherPerCap
skipped ^
HispPerCap
PctBSorMore
PctEmplManu
PctEmplProfServ
PctOccupManu
PctKids2Par
PctYoungKids2Par
NumIlleg
PctIlleg
PctImmigRecent
PctRecImmig5
PctRecImmig10
PctSpeakEnglOnly
PersPerOccupHous
PersPerRentOccHous
PctPersOwnOccup
PctHousLess3BR
HousVacant
MedYrHousBuilt
PctHousNoPhone
OwnOccLowQuart
OwnOccMedVal
RentMedian
MedRentPctHousInc
MedOwnCostPctInc
MedOwnCostPctIncNoMtg
NumStreet
PctSameHouse85
PctSameCity85
PctUsePubTrans


In [ ]:
df['ViolentCrimesPerPop'] = pd.cut(df['ViolentCrimesPerPop'], bins = 3, labels = [1,2,3])

In [ ]:
df['OtherPerCap'] = df['OtherPerCap'].astype(float)

In [ ]:
df['OtherPerCap'] = pd.cut(df['OtherPerCap'], bins=4, labels =cut_labels)

In [ ]:
df.sample(10)

,population,racepctblack,racePctWhite,racePctHisp,agePct12t21,agePct16t24,pctUrban,medFamInc,perCapInc,blackPerCap,OtherPerCap,HispPerCap,PctBSorMore,PctEmplManu,PctEmplProfServ,PctOccupManu,PctKids2Par,PctYoungKids2Par,NumIlleg,PctIlleg,PctImmigRecent,PctRecImmig5,PctRecImmig10,PctSpeakEnglOnly,PersPerOccupHous,PersPerRentOccHous,PctPersOwnOccup,PctHousLess3BR,HousVacant,MedYrHousBuilt,PctHousNoPhone,OwnOccLowQuart,OwnOccMedVal,RentMedian,MedRentPctHousInc,MedOwnCostPctInc,MedOwnCostPctIncNoMtg,NumStreet,PctSameHouse85,PctSameCity85,PctUsePubTrans,ViolentCrimesPerPop
798,1,1,4,1,2,1,1,1,1,2,1,2,1,3,2,3,3,3,1,1,1,1,1,4,2,1,3,2,1,1,1,1,1,1,2,2,3,1,4,4,1,1
975,1,1,4,1,2,2,4,2,2,2,2,1,1,2,2,2,3,3,1,1,1,1,1,4,3,1,4,2,1,2,1,2,2,2,3,3,3,1,4,4,2,1
659,1,1,4,1,2,2,1,1,1,1,1,2,1,4,2,4,3,4,1,1,4,1,1,4,2,2,3,2,1,1,3,1,1,1,2,1,1,1,3,4,1,2
909,3,3,2,1,2,2,4,2,2,1,2,2,2,2,2,2,2,2,3,3,2,1,1,4,2,2,3,3,4,2,2,1,1,1,2,2,2,1,3,3,2,3
1467,1,2,3,1,1,2,4,3,3,2,2,2,3,2,3,1,2,3,1,2,3,4,3,3,2,1,1,3,1,1,1,3,3,3,2,3,3,1,2,3,2,1
1132,1,1,4,1,3,2,4,2,2,4,1,2,2,3,2,3,4,4,1,1,1,1,1,3,2,2,3,2,1,2,1,2,2,2,3,3,3,1,3,3,1,1
1642,1,1,4,1,2,1,4,4,4,3,1,3,3,2,3,1,4,4,1,1,2,1,1,4,3,3,4,1,1,2,1,4,4,4,2,3,2,1,4,3,2,1
1410,1,1,4,1,1,1,4,1,2,2,2,2,1,2,2,2,3,3,1,1,1,1,1,4,1,1,3,4,2,3,1,1,1,2,3,2,1,1,2,3,1,1
300,1,1,4,1,3,3,1,2,1,1,2,2,2,3,3,3,3,3,1,1,3,1,1,4,2,2,2,3,1,3,2,1,1,2,2,2,2,1,2,3,1,1
619,1,1,3,1,2,1,4,4,4,4,4,4,4,2,3,1,4,4,1,1,3,3,3,3,3,3,4,1,1,3,1,4,4,4,3,2,1,1,3,4,1,1


In [ ]:
df = df.drop(["state", "county", "community", "communityname", "fold"], axis=1)
df.sample(5)

,population,racepctblack,racePctWhite,racePctHisp,agePct12t21,agePct16t24,pctUrban,medFamInc,perCapInc,blackPerCap,OtherPerCap,HispPerCap,PctBSorMore,PctEmplManu,PctEmplProfServ,PctOccupManu,PctKids2Par,PctYoungKids2Par,NumIlleg,PctIlleg,PctImmigRecent,PctRecImmig5,PctRecImmig10,PctSpeakEnglOnly,PersPerOccupHous,PersPerRentOccHous,PctPersOwnOccup,PctHousLess3BR,HousVacant,MedYrHousBuilt,PctHousNoPhone,OwnOccLowQuart,OwnOccMedVal,RentMedian,MedRentPctHousInc,MedOwnCostPctInc,MedOwnCostPctIncNoMtg,NumStreet,PctSameHouse85,PctSameCity85,PctUsePubTrans,ViolentCrimesPerPop
1828,1,1,3,1,3,2,4,3,3,2,2,3,4,2,3,1,4,4,1,1,2,3,3,3,3,2,3,3,1,4,1,4,4,4,3,4,2,1,2,2,1,1
1182,1,4,2,1,2,2,4,1,2,1,1,3,2,2,3,2,1,2,1,3,2,1,1,4,2,2,2,2,1,3,3,1,1,1,2,2,2,1,3,3,1,3
6,1,1,4,1,2,1,4,2,2,2,1,2,2,2,3,2,4,4,1,1,1,1,1,3,2,1,3,2,1,2,1,2,2,2,2,2,2,1,4,4,1,1
1600,1,1,4,1,3,1,1,2,1,1,2,2,2,4,2,2,4,4,1,1,1,1,1,4,4,3,3,2,1,2,1,1,1,1,1,1,1,1,3,3,1,1
509,1,1,4,1,3,2,1,1,1,1,1,3,1,2,3,3,3,3,1,1,1,1,1,4,2,2,3,3,1,2,2,1,1,1,3,2,3,1,2,4,1,1


In [ ]:
df.to_csv('/content/drive/MyDrive/ADM Project/descritized_crimedata_labeled_without_info_cols.csv', index=False)